# Kafka

In [ ]:
// list topics
kafka-topics --list --bootstrap-server localhost:9097

// read messages in topic
kafka-console-consumer --bootstrap-server localhost:9097 --topic test-topic --from-beginning

# Spark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = (
    SparkSession
    .builder
    .appName("OpenSEA Pipeline")
    .config("spark.streaming.stopGracefullyonShutdown", True)
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0')
    .config("spark.sql.shuffle.partitions", 4)
    .master("local[*]")
    .getOrCreate()
)

In [ ]:
spark

In [ ]:
# Retrieve data from kafka topic

kafka_df = (
    spark
    .read
    .format("kafka")
    .option("kafka.bootstrap.servers", "os-kafka:9092")
    .option("subscribe", "test-os_event")
    .option("startingOffsets", "earliest")
    .load()
)

In [ ]:
# Show kafka topic schema and messages

kafka_df.printSchema()
kafka_df.show()

In [ ]:
# Parse value from binary to string into kafka_json_df
from pyspark.sql.functions import expr, schema_of_json

kafka_json_df = kafka_df.withColumn("value", expr("cast(value as string)"))

In [ ]:
kafka_json_df.show()

In [ ]:
# Do not delete

json_schema = schema_of_json(kafka_json_df.select('value').head()[0])
json_schema

In [ ]:
df2 = kafka_json_df.withColumn("value", schema)

In [ ]:
from pyspark.sql.functions import from_json,col

streaming_df = kafka_json_df.withColumn("values_json", from_json(col("value"), json_schema)).selectExpr("values_json.*")

In [ ]:
streaming_df.printSchema()
streaming_df.show(truncate=True)

In [ ]:
kafka_topic_2 = kafka_json_df.withColumn("values_json", \
                                        from_json(col("value"), \
                                        json_schema)).selectExpr(
                                            "values_json.event_type", \
                                            "values_json.payload.collection"
)